In [ ]:
# class Passenger:
#     def __init__(self):
#         self.personID = None
#         self.fromAirport = None
#         self.toAirport = None
#         self.preferredDepartureTime = None
#         self.preferredDay = None
#         self.preferredDepartureTimeRange = None  # Converted time to the range of 0 to 1008
#         self.maxWaitTime = None  # Maximum time the passenger can wait


# class Flight:
#     def __init__(self):
#         self.departureTime = None
#         self.arrivalTime = None
#         self.originCity = None
#         self.destinationCity = None
#         self.filledSeats = None
#         self.avgWaitingTime = None
#         self.profit = None


# class Airport:
#     def __init__(self):
#         self.flights = []
#         self.maxFlightsPerHour = None
#         self.flightThresholds = []

In [56]:
# variables
totalFlights = 5
numberOfAirports = 3
totalTimeIntervals = 10


fixedCostPerHourOfFlight = 100
costPerSeatPerHourOfFlight = 50
numberOfSeatsPerFlight = 5

minimumFractionAirplaneFillesd = 0.8

timeFromOneAirportToAnother = 3 # 3 intervals
""

## while alltoing random values to number of people travelling per timeintervale, what is the max valu;e
maxPeoplePerTimeIntervale = 4

In [57]:
import numpy as np

# Create a synthetic dataset
airport_traffic = np.random.randint(0, high=maxPeoplePerTimeIntervale, size=(numberOfAirports, numberOfAirports, totalTimeIntervals))

# Set the diagonal values to 0 since flights from an airport to itself are not considered
# np.fill_diagonal(airport_traffic, 0)

print("Airport Traffic Dataset:")

for i in range(0, len(airport_traffic)):
    for j in range(0, len(airport_traffic)):
        for k in range (0 ,totalTimeIntervals):
            if (i==j):
                airport_traffic[i][j][k] = 0
print(airport_traffic)


Airport Traffic Dataset:
[[[0 0 0 0 0 0 0 0 0 0]
  [0 1 1 2 2 2 2 1 2 1]
  [0 1 0 1 2 2 1 1 1 2]]

 [[2 3 1 2 1 1 2 3 3 0]
  [0 0 0 0 0 0 0 0 0 0]
  [1 0 1 1 1 1 1 2 3 3]]

 [[1 1 2 0 2 1 1 1 3 3]
  [1 2 2 2 0 0 2 0 1 2]
  [0 0 0 0 0 0 0 0 0 0]]]


In [58]:
# ensuring for every airport there is atleast one flight in the stating
while True:
    iniitial_flights_positions = np.random.randint(0, numberOfAirports, size=totalFlights)
    
    # Check if all numbers between 0 and numberOfAirports - 1 are present
    if set(range(numberOfAirports)) <= set(iniitial_flights_positions):
        break

print("Initial flights positions:", iniitial_flights_positions)


Initial flights positions: [2 1 0 1 0]


In [59]:
airport_traffic_notChanged = airport_traffic.copy()
airport_traffic_notChanged

array([[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 1, 2, 2, 2, 2, 1, 2, 1],
        [0, 1, 0, 1, 2, 2, 1, 1, 1, 2]],

       [[2, 3, 1, 2, 1, 1, 2, 3, 3, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 1, 1, 1, 1, 1, 2, 3, 3]],

       [[1, 1, 2, 0, 2, 1, 1, 1, 3, 3],
        [1, 2, 2, 2, 0, 0, 2, 0, 1, 2],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]])

In [60]:
import numpy as np

airport_traffic = airport_traffic_notChanged.copy()
# print(airport_traffic)

flightLocations = iniitial_flights_positions 
flights_status = [{'currently': city,
                   'take_off_time': 0,
                   'landing_time': 0,
                   'from_city': None,
                   'to_city': None,
                   'seats_full': None,
                   'avg_wait_time': None,
                   'profit': None}
                  for city in iniitial_flights_positions]

schedule_array = [[] for _ in range(totalFlights)]
everyPossibleSchedule = []

In [55]:


def getSeatsLeft(airport_traffic, current_airport, dest_airport, time_instant): 
    seats_left = numberOfSeatsPerFlight
    for time_instant in range(0, time_instant + 1):
        if airport_traffic[current_airport][dest_airport][time_instant] > 0:
            demand = airport_traffic[current_airport][dest_airport][time_instant]
            if seats_left > demand: 
                airport_traffic[current_airport][dest_airport][time_instant] = 0
                seats_left -= demand
            else:
                airport_traffic[current_airport][dest_airport][time_instant] = demand - seats_left
                seats_left = 0
    return seats_left

def schedule_flights(time_instant, flights_status, schedule_array, airport_traffic):
    if time_instant == totalTimeIntervals:
        everyPossibleSchedule.append([flight_schedule[:] for flight_schedule in schedule_array])  # Append a copy
        return
    
    flghts_that_can_takeoff = 0
    
    for flight_idx in range(len(flights_status)):
        if flights_status[flight_idx]['currently'] == -1:
            if flights_status[flight_idx]['landing_time'] == time_instant:
                flights_status[flight_idx]['currently'] = flights_status[flight_idx]['to_city']
        if flights_status[flight_idx]['currently'] == -1:
            continue
        
        flight_old = flights_status[flight_idx].copy()
            
        current_airport = flights_status[flight_idx]['currently']
        for dest_airport in range(numberOfAirports):
            if dest_airport == current_airport:
                continue
            
            seatsLeft = getSeatsLeft(airport_traffic, current_airport, dest_airport, time_instant)
            # print(airport_traffic)
            # print("Seats filled:", seatsLeft)
            if seatsLeft > minimumFractionAirplaneFillesd * numberOfSeatsPerFlight:
                continue
            
            flights_status[flight_idx]['currently'] = -1
            flights_status[flight_idx]['take_off_time'] = time_instant
            flights_status[flight_idx]['landing_time'] = time_instant + timeFromOneAirportToAnother
            flights_status[flight_idx]['from_city'] = current_airport
            flights_status[flight_idx]['to_city'] = dest_airport
            flights_status[flight_idx]['seats_full'] = numberOfSeatsPerFlight - seatsLeft
            flights_status[flight_idx]['profit'] =   costPerSeatPerHourOfFlight * (numberOfSeatsPerFlight - seatsLeft) - fixedCostPerHourOfFlight
            
            schedule_array[flight_idx].append(flights_status[flight_idx].copy())
            
            schedule_flights(time_instant + 1, flights_status, schedule_array, airport_traffic)
            
            flights_status[flight_idx] = flight_old
            schedule_array[flight_idx].pop()
            
            schedule_flights(time_instant + 1, flights_status, schedule_array, airport_traffic)

schedule_flights(0, flights_status, schedule_array, airport_traffic)

# Print everyPossibleSchedule to check the output
# print(everyPossibleSchedule)


In [ ]:
airport_traffic

In [ ]:
iniitial_flights_positions

In [ ]:
# schedule_flights(0,flights_status, schedule_array, airport_traffic)    

In [ ]:
everyPossibleSchedule[0]

In [ ]:
len(everyPossibleSchedule)

In [ ]:
everyPossibleSchedule[0]

In [ ]:
profits = []
for schedule in everyPossibleSchedule:
    profit = 0
    for flight_0 in schedule[0]:
        profit = profit + flight_0['profit']
    for flight_1 in schedule[1]:
        profit = profit + flight_1['profit']
    for flight_2 in schedule[2]:
        profit = profit + flight_2['profit']
    for flight_3 in schedule[3]:
        profit = profit + flight_3['profit']
    for flight_4 in schedule[4]:
        profit = profit + flight_4['profit']
    profits.append(profit)
print(profits)
    

In [ ]:
a  = [1,2,3,4,5,6,7,8,9,10]

for i in a :
    i = i + 1
    
a

In [25]:
import copy

In [61]:
import numpy as np

airport_traffic = airport_traffic_notChanged.copy()
# print(airport_traffic)

flightLocations = iniitial_flights_positions 
flights_status = [{'currently': city,
                   'take_off_time': 0,
                   'landing_time': 0,
                   'from_city': 0,
                   'to_city': 0,
                   'seats_full': None,
                   'avg_wait_time': None,
                   'profit': None}
                  for city in iniitial_flights_positions]

schedule_array = [[] for _ in range(totalFlights)]
everyPossibleSchedule = []

def getSeatsLeft(airport_traffic, current_airport, dest_airport, time_instant): 
    seats_left = numberOfSeatsPerFlight
    for time_instant in range(0, time_instant + 1):
        if airport_traffic[current_airport][dest_airport][time_instant] > 0:
            demand = airport_traffic[current_airport][dest_airport][time_instant]
            if seats_left > demand: 
                airport_traffic[current_airport][dest_airport][time_instant] = 0
                seats_left -= demand
            else:
                airport_traffic[current_airport][dest_airport][time_instant] = demand - seats_left
                seats_left = 0
    return seats_left

def schedule_flights(time_instant, flights_status, schedule_array, airport_traffic):
    if time_instant == 2:
        everyPossibleSchedule.append([flight_schedule[:] for flight_schedule in schedule_array])  # Append a copy
        # everyPossibleSchedule.append(schedule_array)
        return
    
    flights_that_can_takeoff = []
    
    for flight_idx in range(len(flights_status)):
        this_flight_can_takeoff = []
        
        if flights_status[flight_idx]['currently'] == -1:
            if flights_status[flight_idx]['landing_time'] == time_instant:
                flights_status[flight_idx]['currently'] = flights_status[flight_idx]['to_city']
        if flights_status[flight_idx]['currently'] == -1:
            this_flight_can_takeoff.append(flights_status[flight_idx])
            continue
        
        
        current_airport = flights_status[flight_idx]['currently']
        
         ## for a particular flight with fligh id, how many flights can take off including the one in which it doesnt take off
         
        to_city = flights_status[flight_idx]['to_city'] 
        flights_status[flight_idx]['to_city'] = flights_status[flight_idx]['from_city']
        this_flight_can_takeoff.append(flights_status[flight_idx].copy())
        
        flights_status[flight_idx]['to_city'] = to_city
        
        
        for dest_airport in range(numberOfAirports):
            if dest_airport == current_airport:
                continue
    
            seatsLeft = getSeatsLeft(airport_traffic, current_airport, dest_airport, time_instant)
            # print(airport_traffic)
            # print("Seats filled:", seatsLeft)
            if seatsLeft > minimumFractionAirplaneFillesd * numberOfSeatsPerFlight:
                continue
            
            flight_that_can_takeoff = {}
            flight_that_can_takeoff['currently'] = current_airport
            flight_that_can_takeoff['from_city'] = current_airport
            flight_that_can_takeoff['to_city'] = dest_airport
            flight_that_can_takeoff['seats_full'] = numberOfSeatsPerFlight  - seatsLeft
            flight_that_can_takeoff['take_off_time'] = time_instant
            flight_that_can_takeoff['current_airport'] = current_airport
            flight_that_can_takeoff['landing_time'] = time_instant + timeFromOneAirportToAnother
            flight_that_can_takeoff['profit'] = costPerSeatPerHourOfFlight * (numberOfSeatsPerFlight - seatsLeft) - fixedCostPerHourOfFlight
            
            this_flight_can_takeoff.append(flight_that_can_takeoff.copy())
            
            # flights_status[flight_idx]['currently'] = -1
            # flights_status[flight_idx]['take_off_time'] = time_instant
            # flights_status[flight_idx]['landing_time'] = time_instant + timeFromOneAirportToAnother
            # flights_status[flight_idx]['from_city'] = current_airport
            # flights_status[flight_idx]['to_city'] = dest_airport
            # flights_status[flight_idx]['seats_full'] = numberOfSeatsPerFlight - seatsLeft
            # flights_status[flight_idx]['profit'] =   costPerSeatPerHourOfFlight * (numberOfSeatsPerFlight - seatsLeft) - fixedCostPerHourOfFlight\
            # schedule_array[flight_idx].append(flights_status[flight_idx].copy())
            # schedule_flights(time_instant + 1, flights_status, schedule_array, airport_traffic)
            
            # flights_status[flight_idx] = flight_old
            # schedule_array[flight_idx].pop()
            
            # schedule_flights(time_instant + 1, flights_status, schedule_array, airport_traffic)
        flights_that_can_takeoff.append(this_flight_can_takeoff.copy())
    
    ## for a particular time instant, choosing which all flights to schedule and which doesnt, including every combination
    
    print("flightsc that be takeoff" , flights_that_can_takeoff)
    for flight_0 in flights_that_can_takeoff[0]:
        for flight_1 in flights_that_can_takeoff[1]:
            for flight_2 in flights_that_can_takeoff[2]:
                for flight_3 in flights_that_can_takeoff[3]:
                    for flight_4 in flights_that_can_takeoff[4]:
                        flight_0_Added = False
                        flight_1_Added = False
                        flight_2_Added = False
                        flight_3_Added = False
                        flight_4_Added = False
                        
                        
                        if(flight_0['currently'] != -1 and flight_0['to_city'] != flight_0['from_city']):
                            flight_0_Added = True
                            flight_0['currently'] = -1
                            schedule_array[0].append(flight_0.copy())
                            
                        if(flight_1['currently'] != -1 and flight_1['to_city'] != flight_1['from_city']):
                            flight_1_Added = True
                            flight_1['currently'] = -1
                            schedule_array[1].append(flight_1.copy())
                            
                        if(flight_2['currently'] != -1 and flight_2['to_city'] != flight_2['from_city']):
                            flight_2_Added = True
                            flight_2['currently'] = -1  
                            schedule_array[2].append(flight_2.copy())
                            
                        if(flight_3['currently'] != -1 and flight_3['to_city'] != flight_3['from_city']):
                            flight_3_Added = True
                            flight_3['currently'] = -1  
                            schedule_array[3].append(flight_3.copy())
                            
                        if(flight_4['currently'] != -1 and flight_4['to_city'] != flight_4['from_city']):
                            flight_4_Added = True
                            flight_4['currently'] = -1
                            schedule_array[4].append(flight_4.copy())
                            
                            print("timestamp" , time_instant , "schedule_array", schedule_array)
                            new_flights_status = copy.deepcopy(flights_status)
                            new_schedule_array = copy.deepcopy(schedule_array)
                                
                            schedule_flights(time_instant + 1, new_flights_status, new_schedule_array, airport_traffic)
                        
                        
                        if(flight_0_Added):
                            schedule_array[0].pop()
                        if(flight_1_Added):
                            schedule_array[1].pop()
                        if(flight_2_Added):
                            schedule_array[2].pop()
                        if(flight_3_Added):
                            schedule_array[3].pop()
                        if(flight_4_Added):
                            schedule_array[4].pop()
                        
        

schedule_flights(0, flights_status, schedule_array, airport_traffic)

# Print everyPossibleSchedule to check the output
# print(everyPossibleSchedule)


flightsc that be takeoff [[{'currently': 2, 'take_off_time': 0, 'landing_time': 0, 'from_city': 0, 'to_city': 0, 'seats_full': None, 'avg_wait_time': None, 'profit': None}, {'currently': 2, 'from_city': 2, 'to_city': 0, 'seats_full': 1, 'take_off_time': 0, 'current_airport': 2, 'landing_time': 3, 'profit': -50}, {'currently': 2, 'from_city': 2, 'to_city': 1, 'seats_full': 1, 'take_off_time': 0, 'current_airport': 2, 'landing_time': 3, 'profit': -50}], [{'currently': 1, 'take_off_time': 0, 'landing_time': 0, 'from_city': 0, 'to_city': 0, 'seats_full': None, 'avg_wait_time': None, 'profit': None}, {'currently': 1, 'from_city': 1, 'to_city': 0, 'seats_full': 2, 'take_off_time': 0, 'current_airport': 1, 'landing_time': 3, 'profit': 0}, {'currently': 1, 'from_city': 1, 'to_city': 2, 'seats_full': 1, 'take_off_time': 0, 'current_airport': 1, 'landing_time': 3, 'profit': -50}], [{'currently': 0, 'take_off_time': 0, 'landing_time': 0, 'from_city': 0, 'to_city': 0, 'seats_full': None, 'avg_wait

In [62]:
len(everyPossibleSchedule)

0

In [63]:
everyPossibleSchedule[23]

IndexError: list index out of range

In [41]:
profits = []
for schedule in everyPossibleSchedule:
    profit = 0
    for flight_0 in schedule[0]:
        if(flight_0["profit"]):
            profit = profit + flight_0['profit']
    for flight_1 in schedule[1]:
        if(flight_1["profit"]):
            profit = profit + flight_1['profit']
    for flight_2 in schedule[2]:
        if(flight_2["profit"]):
            profit = profit + flight_2['profit']
    for flight_3 in schedule[3]:
        if(flight_3["profit"]):
            profit = profit + flight_3['profit']
    for flight_4 in schedule[4]:
        if(flight_4["profit"]):
            profit = profit + flight_4['profit']
    profits.append(profit)
print(profits)
    
        

[0, 0, 50, -50, -50, 0, 0, 0, 50, -50, -50, 0, 50, 50, 100, 0, 0, 50, -50, -50, -50, -100, -100, 0, -50, -50, -50, -100, -100, 50, 0, 0, 0, -50, -50]


In [42]:
everyPossibleSchedule[5]

[[{'currently': 2,
   'take_off_time': 0,
   'landing_time': 0,
   'from_city': 0,
   'to_city': 0,
   'seats_full': None,
   'avg_wait_time': None,
   'profit': None},
  {'currently': 2,
   'take_off_time': 0,
   'landing_time': 0,
   'from_city': 0,
   'to_city': 0,
   'seats_full': None,
   'avg_wait_time': None,
   'profit': None}],
 [{'currently': 2,
   'take_off_time': 0,
   'landing_time': 0,
   'from_city': 0,
   'to_city': 0,
   'seats_full': None,
   'avg_wait_time': None,
   'profit': None},
  {'currently': 2,
   'take_off_time': 0,
   'landing_time': 0,
   'from_city': 0,
   'to_city': 0,
   'seats_full': None,
   'avg_wait_time': None,
   'profit': None}],
 [{'currently': 0,
   'take_off_time': 0,
   'landing_time': 0,
   'from_city': 0,
   'to_city': 0,
   'seats_full': None,
   'avg_wait_time': None,
   'profit': None},
  {'currently': -1,
   'from_city': 0,
   'to_city': 1,
   'seats_full': 1,
   'take_off_time': 1,
   'current_airport': 0,
   'landing_time': 4,
   'pro

In [ ]:
everyPossibleSchedule